<a href="https://colab.research.google.com/github/marceauguiot/q-a/blob/master/NLP_Q%26A_without_output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**NLP Project : Q & A system**

> Group 6 : Maud Galmiche, Thibaut Nicot, Christophe Arendt, Sebastien Bordonnat, Marceau Guiot



![NLP Project : Q & A system](https://www.escpeurope.eu/sites/default/files/default_images/default-picture-news-md.jpg)

In this NLP project, we are asked to predict the best answer if any exists. In the dataset provided, there is not always a best answer for each question. Another issue is that there are some comments upon some answers. In a first time, we will build a model only upon answers. 

For this project, we decided to work on google colab. In fact as students, we did not have access to a GPU with our laptop so we decided to use the one freely provided by google colab. To colaborate, we decided to write all our code on a notebook. Since this project has non intent to be put in production, we think that would be the most convenient format to work on it.

This project will be strucutred in 3 main parts : 


1.   Data preprocessing
2.   Data vizualization
3.   Model training using Flair

In the data preprocessing, we will discuss a little bit about the data and how we can import them. Quality of data is often a big deal for data scientists. In the data provided by the dataset, some separators can be misunderstood by regular fonctions normally used to import csv as the one provided by pandas. That is why, in this part, we will try to figure out how we can import the data provided.

In data vizualization, we will try to figure out how our data are strctured. We will also have a close look on how our labels are balanced ( the number of best answers compared to the total amount of answers). We will also give close attention to the length of each answer since it can cause some memory issue when we are doing the embedding. 



# Data preprocessing

## Importing libraries

In [0]:
# General libraries
from textblob import TextBlob
import pandas as pd
import nltk
import torch
import string
import pickle
import json
import tensorflow as tf
import os
import numpy as np
import re
import string
from hyperopt import hp
from tqdm import tqdm 
import cufflinks as cf
from datetime import datetime, timedelta
from pathlib import Path

# From scikit learn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

#From nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize, TweetTokenizer
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import TreebankWordTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.corpora import Dictionary



#Google colab
from google.colab import drive

## Importing drive

In order to have access to the dataset provided, we mount our drive containing the csv file. This operation allow us to import files from the drive but also to write and save files on the google drive. We just have to copy the authorization code to access the drive.

In [0]:
# This will prompt for authorization.
drive.mount('/content/drive')

Once the drive is mounted, we can change our working directory. 

In [0]:
# Change working directory
%cd "/content/drive/My Drive/Q & A/"

##Importing data 

As explained before, the data export-forum_en.csv are some format issues. In spite of this issues, we try to force the importation using pandas to have a first glance at the data.

### With pandas

In [0]:
columns = ["id",
"question_answer_or_comment" , 
"is_best_answer",
"topic_id",
"parent_id",
"votes",
"titre",
"message",
"member", 
"category", 
"state",
 "is_solved", 
"num_answers",
"country", 
"date",
"last_answer_date", 
"auteur_crc", 
"visits"]

In [0]:
data = pd.read_csv("export-forums_en.csv", error_bad_lines = False, names = columns)

In [0]:
data.head(10)

Even if at first sight, the importation seems to have worked, we can see after a quick analysis that we have facing some issues. In fact, after a few lines, the importation process shifts all the columns, damaging the data. 

In [0]:
data[data["is_best_answer"] == ' update data according to official site'].head()

### With a customized method

In a first time, we decided to work with the pandas importation excluding the data corrupted with a simple filtered. Since, our NLP professor M. Anh-Phuong decided to give us another method to import the whole dataset, we switched to his method.

In [0]:
txt_path = "export-forums_en.csv"
entity_path = "export-forums_en.pickle"
csv_path = "export-forums_en_TA.csv"
data_path = ""


def format_entities():
    '''
    Read the raw data, format the list of entities, serialize them.
    '''

    def build_entities(txt_path, max_entities=None):
        '''
        Return a list of structured entities from raw txt file.
        '''
        # Read text file.
        with open(txt_path, 'r', encoding='utf8') as f:
            # Entities and current entity.
            entities, entity = [], {}
            # Entity values might be split over lines
            field_counter = 0
            # Process lines
            for line in f:
                # Prepare line
                line = line.replace("\\N", '"unkwown"')
                # Char start for extracted value.
                char_start = 1
                # Find values separators
                field_index = [m.start() for m in re.finditer('","', line)]
                # Browse value separators.
                for index in field_index:
                    # Extract in between value.
                    value = line[char_start:index]
                    # Update start index.
                    char_start = index + 3
                    # Update field counter.
                    field_counter += 1
                    # Update entity value.
                    try:
                        entity[columns[field_counter-1]] += value
                    except KeyError:
                        entity[columns[field_counter-1]] = value
                    except IndexError:
                        entity = {}
                        field_counter = 0
                # Content string is split.
                if field_counter == 7 and len(field_index) > 0:
                    entity[columns[7]] = line[field_index[-1]:]
                    continue
                # Next content string.
                if field_counter == 7 and len(field_index) == 0:
                    entity[columns[7]] += line
                    continue
                # Next entity.
                if len(entity) == 17:
                    field_counter = 0
                    entities.append(entity)
                    entity = {}
                    if max_entities is not None:
                        if len(entities) > max_entities:
                            return entities
        return entities

    # Write entities on disk.
    with open(entity_path, 'wb') as f:
        pickle.dump(build_entities(txt_path=txt_path, max_entities=None), f)


def entities_to_csv():
    '''
    Format entities to csv.
    '''
    with open(entity_path, 'rb') as obj:
        entities = pickle.load(obj)
    x = pd.DataFrame(entities, index = None)
    x.to_csv(csv_path, index = None)

In [0]:
format_entities()
entities_to_csv()

### Checking imported data

Once the csv is saved, we can easily checked that everything went well transforming our target variable into a set.

In [0]:
data = pd.read_csv("export-forums_en_TA.csv")

In [0]:
data.head()

In [0]:
set(data["is_best_answer"])

Even if everything seems to went well during the importation, we are still facing a few issues. In fact the two columns date and last_answer_date are in second elapsed since the 1st January 1970. In order to have an appropriate view of this data we transform these two columns in proper date.

##Convert time to date

In order to know in which fomat the time is represented we search on the web the forum related to the first question :  [click-here](https://ccm.net/forum/affich-4-windows-vista-to-xp-downgrading-reformat).  Once this is done, we just have to be sure that every columns is in the right format.

In [0]:
date = datetime(1970,1,1) # January 1st, 1904 at midnight

delta = timedelta(seconds = 1207882212)

newdate = date + delta
print(newdate)


In [0]:
def new_date(seconds):
  date = datetime(1970,1,1) # January 1st, 1904 at midnight

  delta = timedelta(seconds = seconds)

  newdate = date + delta
  return(newdate)
  

In [0]:
#Checking the format of the both columns
data['date'] = pd.to_numeric(data['date'], errors='coerce')
data['last_answer_date'] = pd.to_numeric(data['last_answer_date'], errors='coerce')

In [0]:
data['date'] = data['date'].apply(lambda x: new_date(x))

In [0]:
data['last_answer_date'] = data['last_answer_date'].apply(lambda x: new_date(x))

In [0]:
data.head()

In [0]:
data['titre'] = data['titre'].apply(lambda x: str(x))
data['message'] = data['message'].apply(lambda x: str(x))

## Removing comments

As explained before, we start by building a model without considering the comments.

In [0]:
data = data[ data.question_answer_or_comment != "C"]

In [0]:
len(data)

In [0]:
data_a = data[ ["titre" ,"message", "is_best_answer"]]

##Text cleaning

### Stopwords

One major issue we are facing with the dataset provided is the fact that this one is multilingual. This cause trouble for both cleaning text and the embedding. In fact, the library *nltk* provided some stopwords to clean the text but those are specific to one language. What can be done here is to build a model to detect the language of each question. Another soltuion, less time consuming is to concatenate the list of stopwords for each language. This [Git-hub](https://github.com/6/stopwords-json) provides a json file gathering stopwords from multiple languages.

In [0]:
#!git clone https://github.com/6/stopwords-json.git

In [0]:
with open('stopwords-json/stopwords-all.json') as json_file:  
    stop_words = json.load(json_file)

In [0]:
"the" in stop_words

In [0]:
stop_words = list(stop_words.values())
stop_words = set([item for sublist in stop_words for item in sublist])

In [0]:
"the" in set(stop_words)

Here we are using a set in stead of a list to decrease the computing time. Since we have now defined our stopwords we can define a function to clean the text. First we have to choose the variable on which we will apply the function.

Since we want the information on both the question and the message, we decide here to concatenate the both columns. Once this will be done, we will be able to get rid off the titre of each question since it will also be contained in the message.

The data provided might certainly come from scrapping. That is why we need to get clean all the HTML tags first. Once this is done we can replace all the punctuations characters with spaces and convert all the words to lower case. At theend we can apply our tokenizer on the text. Here we decided to use the *TreebankWordTokenizer*. At the end of the day, we check of course if the word is a stopowrd before adding it to our message.

In [0]:
tokenizer = TreebankWordTokenizer()

def clean_text(text):
    """
    Applies some pre-processing on the given text.

    Steps :
    - Removing HTML tags
    - Removing punctuation
    - Lowering text
    """
    # remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # remove the characters [\], ['] and ["]
    text = re.sub(r"\\", "", text)    
    text = re.sub(r"\'", "", text)    
    text = re.sub(r"\"", "", text)
    text = re.sub(r"br", "", text)
    text = re.sub(r"quot", "", text)
    text = re.sub(r"http", "", text)
    
    # remove other characters
    text = re.sub('^(.*http)',"", text)
    
    # convert text to lowercase
    text = text.strip().lower()
    
    # replace punctuation characters with spaces
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((c, " ") for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)
    
    #tokenize
    token = tokenizer.tokenize(text)
    
    # filter out stop words
    #stop_words = set(stopwords.words('english'))
    
    words = [w for w in token if not w in stop_words]
    words = " ".join(words)

    return words

As explained before, we start with concatenating the both columns message and titre.

In [0]:
tqdm.pandas()
data_a['message'] = data_a[['titre', 'message']].apply(lambda x: ''.join(x), axis=1)

Once the concatenation is done, we can apply our function to clean the text.

In [0]:
data_a['titre'] = data_a['titre'].apply(lambda x: clean_text(x))

In [0]:
data_a['message'] = data_a['message'].apply(lambda x: clean_text(x))

In [0]:
data_a.head()

In [0]:
data_a = data_a[["message", "is_best_answer"]]

In [0]:
data_a.to_csv("data_cleaned.csv", index = None)

# Data visualization

In [0]:
data = pd.read_csv("data_cleaned.csv")

In [0]:
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

One of the most important thing to do here is to check how balanced are our label in the target variables. Of course, as we have expected, the best answers are really scarce in our dataset. We have to keep that in our mind when will analyse the results of our model.

In [0]:
from plotly.offline import iplot
import plotly.graph_objs as go

def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
  init_notebook_mode(connected=True)

In [0]:
enable_plotly_in_cell()
fig = data['is_best_answer'].iplot(
    kind='hist',
    xTitle='rating',
    linecolor='black',
    yTitle='count',
    title='Review best answers Distribution',
     )


In order to see which are the most common words, we build a bag of words. Once this bag is build, we can sort it in order to have the most frequent words.

In [0]:
data = data.fillna(' ')
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_words(data['message'], 20)
for word, freq in common_words:
    print(word, freq)

Since we are facing some trouble when we want to display our plot, we build a little function to force colab to authorize plotly to show in the active cell.

In [0]:
enable_plotly_in_cell()
df1 = pd.DataFrame(common_words, columns = ['ReviewText' , 'count'])
df1.groupby('ReviewText').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black', title='Top 20 words in review after removing stop words')


Of course here we see that the most common words are english and related to IT. 

In [0]:
data['review_len'] = data['message'].astype(str).apply(len)
data['word_count'] = data['message'].apply(lambda x: len(str(x).split()))

Analysing the distribution of the length of the answers according if its best answer or not can also be interesting for us. In fact we can expect, that long and detailed answers are more likely to be labelled as best answers. 

In [0]:
y0 = data.loc[data['is_best_answer'] == 1]['review_len']
y1 = data.loc[data['is_best_answer'] == 0]['review_len']


trace0 = go.Box(
    y=y0,
    name = 'Best answers',
    marker = dict(
        color = 'rgb(214, 12, 140)',
    )
)
trace1 = go.Box(
    y=y1,
    name = 'Others',
    marker = dict(
        color = 'rgb(0, 128, 128)',
    )
)
data_plot= [trace0, trace1]
layout = go.Layout(
    title = "Review length Boxplot of best answers"
)
enable_plotly_in_cell()
fig = go.Figure(data=data_plot,layout=layout)
iplot(fig, filename = "Review Length Boxplot of best answers")

The results show that the length does not seem to have a significance impact on the label. In fact both distribution for best answers and other answers are really similar.

In order to prevent memory issue when will do the embedding on our message, we check the length of the messages. In fact, a message that is too long can lead to a lack of RAM and can shut down our google colab session.

In [0]:
enable_plotly_in_cell()
data['review_len'].iplot(
    kind='hist',
    bins=100,
    xTitle='review length',
    linecolor='black',
    yTitle='count',
    title='Review Text Length Distribution')

In [0]:
enable_plotly_in_cell()
data['word_count'].iplot(
    kind='hist',
    bins=100,
    xTitle='word count',
    linecolor='black',
    yTitle='count',
    title='Review Text Word Count Distribution')

# Model training with Flair

In [0]:
data = pd.read_csv("data_cleaned.csv")

##Formatting data into Fasttext

In order to train our model, we have to put our data in the Fast Text format. Fast Text is an open-source, free, lightweight library that allows users to learn text representation and text classifiers. Here, we will not train our model through Fast Text. We will build our model with the library Flair (presented below). Since Flair is using Fast Text corpuses, we need to adopt this format. 

![Fast text](https://fasttext.cc/img/ogimage.png)

In [0]:
data = pd.read_csv("data_cleaned.csv")

data["is_best_answer"] = data["is_best_answer"].replace(1, "best")
data["is_best_answer"] = data["is_best_answer"].replace(0, "not_best")


data = data[['is_best_answer', 'message']].rename(columns={"is_best_answer":"label", "message":"text"})

# Cuda issues
data = data[1:10000]

data['label'] = '__label__' + data['label'].astype(str)
data.iloc[0:int(len(data)*0.8)].to_csv('data_flair/train.csv', sep='\t', index = False, header = False)
data.iloc[int(len(data)*0.8):int(len(data)*0.9)].to_csv('data_flair/test.csv', sep='\t', index = False, header = False)
data.iloc[int(len(data)*0.9):].to_csv('data_flair/dev.csv', sep='\t', index = False, header = False);


A fast text corpus is defined by a train, a test and a dev dataset. To reduce the computing time and to avoid memory issues, we only usethe first 10 000 raws of our dataset for the moment. 

## Using Flair with gpu

Flair is python library created by Zalando research team. This library draws a very precise state of the art in NLP and permits to reach good score on regular problems.


*   **A powerful NLP library** : Flair allows us to apply the state-of-the-art natural language processing to our text
*   **Multilingual** : Flair supports multilingual text. This is very useful in our case since the data are multilingual
* **A text embedding library** : Flair allows us to use and combine different embeddings. In this project we used Flair embedding combined with BERT embedding.
* **A Pytorch NLP Framework** : the framework is directly build on Pytorch




![Flair](https://avatars0.githubusercontent.com/u/30869512?s=400&v=4)



You can find the Git-hub link to Flair library [here](https://github.com/zalandoresearch/flair).

Since Flair is a really specific library, it is not by default installed on our Virtual Machine provided by Google Colab. That is why we have to install it each we connect to our virtual machine : 

In [0]:
!pip install flair

Once it is installed, we can load the function we will need in our project : 

In [0]:
# From flair
from flair.data_fetcher import NLPTaskDataFetcher, NLPTask
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings, BertEmbeddings, StackedEmbeddings, DocumentPoolEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer 
from flair.hyperparameter.param_selection import SearchSpace, Parameter
from flair.visual.training_curves import Plotter
from flair.hyperparameter.param_selection import TextClassifierParamSelector, OptimizationValue
from flair.data_fetcher import NLPTaskDataFetcher
#from flair.data import TaggedCorpus

Once flair is installed, we check that our device is well settled. We specify here that we want our tensorflow backend to use the GPU provided by Google Colab.![](https://cdn-images-1.medium.com/max/1200/0*c0dbic0TZRh9ILrx.jpg)

In [0]:
tf.test.gpu_device_name()

In [0]:
import flair
flair.device = torch.device('cuda:0')

## Importing Corpus

Since the device is now settled, we can start by importing our Corpus here. Flair is only using Fats text corpus, that is why the previous step of formatting our data was crucial. 

In [0]:
# Change working directory
%cd "/content/drive/My Drive/Q & A/"

In [0]:
# use your own data path
data_folder = Path('data_flair')

# load corpus containing training, test and dev data
TaggedCorpus = NLPTaskDataFetcher.load_classification_corpus(data_folder,
                                                                     test_file='test.csv',
                                                                     train_file='train.csv',
                                                                     dev_file='dev.csv',)

To avoid memory issues, we just check that every reponse is not too long. We also do a small analysis using some descriptive statistics on our corpus.

In [0]:
max_tokens = 512
corpus._train = [x for x in corpus.train if len(x) < max_tokens]
corpus._dev = [x for x in corpus.dev if len(x) < max_tokens]
corpus._test = [x for x in corpus.test if len(x) < max_tokens]
stats = corpus.obtain_statistics()
print(stats)

## Embedding

Flair provides different type of embedding. 


*  **Classic Word Embeddings**
*   **Character Embeddings**
* **Byte Pair Embeddings**
* **Stacked Embeddings**
* **Flair Embeddings**
* **Bert Embeddings**


Here we will combine BERT and Flair embedding. We decided to use Document Embeddings in stead of Word Embedding. Word Embeddings give us one embedding for individual words unlike document embeddings give us embeddings for an entire text. 


Different methods are available for document Embeddings : 


*   **Pooling** : This method calculates a pooling operation over all word embeddings in a documents. Here we take the *mean* off all the words in the answer. We can also use the operations *min* or *max*.
*   **RNN** : Flair also provides a method based on RNN. The default RNN is a GRU-type but we can also use LSTM.

In our project we use a document embedding pooling with Flair Embeddings combined with BERT Embeddings.



In [0]:
# 2. create the label dictionary
label_dict = corpus.make_label_dictionary()


# init Flair embeddings
flair_forward_embedding = FlairEmbeddings('multi-forward-fast')
flair_backward_embedding = FlairEmbeddings('multi-backward-fast')

# init multilingual BERT
bert_embedding = BertEmbeddings('bert-base-multilingual-cased') 

#init Glove embedding
glove_embedding = WordEmbeddings('glove')

In [0]:
# now create the StackedEmbedding object that combines all embeddings
document_embeddings = DocumentPoolEmbeddings(
    embeddings=[flair_forward_embedding,
                flair_backward_embedding,
                bert_embedding
               ])

#embedding gloobe
document_embeddings_glove = DocumentPoolEmbeddings(embeddings = [glove_embedding])

Once our Embedding is settled,  we can use the Textclassifier built by Flair : [text_classification_model.py](https://github.com/zalandoresearch/flair/blob/master/flair/models/text_classification_model.py)

In [0]:
# 5. create the text classifier
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict, multi_label=False)

# 6. initialize the text classifier trainer
trainer = ModelTrainer(classifier, corpus)

## Training

Once our document embedding is done we can train the model. We have to choose the parameters of our models : 



*   **Learning rate** : The amount of change to the model during each step of this search process, or the step size. The learning rate controls how quickly or slowly a neural network model learns a problem. To set up the learning rate we will use the function find_learning_rate provided by flair.
*   **Mini batch size** :  batch size  the number of samples to work through before updating the internal model parameters
*  **Hidden size** : number of hidden layer in the LSTM
*  **Maximum of epoch** : epoch defines the number times that the learning algorithm will work through the entire training dataset
*  **Embeddings** : we can train our model with different embeddings
*  **RNN Layers** : number of layers in our Recurrent Neural Network
* **Dropout** :  Dropout is a regularization method where input and recurrent connections to LSTM units are probabilistically excluded from activation and weight updates while training a network. This has the effect of reducing overfitting and improving model performance.





In [0]:
# 8. start the training
trainer.train("data_flair",
              learning_rate=0.1,
              mini_batch_size=8,
              embeddings_in_memory = False,
              max_epochs=150, 
              checkpoint = True)

We are facing an issu with Google Colab. In fact, Google Colab close the connection to the virtual machine to fight against coder who are using google colab to mine bitcoin. To solve this problem, we save the model after each epoch with the parameter *checkpoint = True*. Thus, even if we are disconnected from google colab we can still resume the training after. 

In [0]:
trainer = ModelTrainer.load_from_checkpoint(Path("data_flair/checkpoint.pt"), 'TextClassifier', corpus)

trainer.train("data_flair",
              learning_rate=0.1,
              mini_batch_size=8,
              embeddings_in_memory = False,
              max_epochs=150,
              checkpoint=True)

## Loading best model

Once our model is trained we can load the best model :

In [0]:
trainer = ModelTrainer.load_from_checkpoint(Path("data_flair/best-model.pt"), 'TextClassifier', corpus)

## Learning rate

To define the best learning rate, we can use this fuction provided by NLP. This graph represents the loss for different learning rate. We have here to choose the learning rate which maximises the loss.

In [0]:
# 7. find learning rate
learning_rate_tsv = trainer.find_learning_rate('data_flair',
                                               'learning_rate.tsv',
                                              mini_batch_size=32)

# 8. plot the learning rate finder curve
plotter = Plotter()
plotter.plot_learning_rate(learning_rate_tsv)


In [0]:
from IPython.display import Image

Image('data_flair/learning_rate.png')

## Hyper parameters selection

In order to select the best parameters for our model, Flair also provides an optimizer. This optimizer his highly time consuming but can be really convenient to find the best parameters for our model. 

In [0]:
# define your search space
search_space = SearchSpace()
search_space.add(Parameter.EMBEDDINGS, hp.choice, options=[
  [WordEmbeddings('glove')],
   [FlairEmbeddings('multi-forward-fast'), FlairEmbeddings('multi-backward-fast')]
])
search_space.add(Parameter.HIDDEN_SIZE, hp.choice, options=[32, 64, 128])
search_space.add(Parameter.RNN_LAYERS, hp.choice, options=[1, 2])
search_space.add(Parameter.DROPOUT, hp.uniform, low=0.0, high=0.5)
search_space.add(Parameter.LEARNING_RATE, hp.choice, options=[0.05, 0.1, 0.15, 0.2])
search_space.add(Parameter.MINI_BATCH_SIZE, hp.choice, options=[8, 16, 32])

In [0]:
# create the parameter selector
param_selector = TextClassifierParamSelector(
    corpus, 
    False, 
    'data_flair', 
    'lstm',
    mini_batch_size=1,
    max_epochs=150, 
    training_runs=3,
    optimization_value=OptimizationValue.DEV_SCORE
)

# start the optimization
param_selector.optimize(search_space, max_evals=3)

In [0]:
# 8. plot training curves (optional)
from flair.visual.training_curves import Plotter
plotter = Plotter()
plotter.plot_training_curves('data_flair/loss.tsv')
plotter.plot_weights('data_flair/weights.txt')

# END------------------------